## **Visualizating bifurcations given by Mimics in .txt**

### **Imports and Installs**

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
import plotly.graph_objects as go
import networkx as nx
from scipy.spatial import cKDTree
import string

### **Google Drive connection**

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Upload files**

### **Functions**

In [ ]:
def generate_file_paths(case_name):
  rca = f'/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/{case_name}/rca_centerline.txt'
  lca = f'/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/{case_name}/lca_centerline.txt'
  return rca, lca

In [ ]:
def generate_file_paths_letters(case_name):
  rca = f'/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/{case_name}/rca_centerline_letters.txt'
  lca = f'/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/{case_name}/lca_centerline_letters.txt'
  return rca, lca

In [ ]:
def segments_to_letters(input_path, output_path):
    """
    Replaces segment numbers (1-17) with letters (A-Q) in a given text file.

    Args:
        input_path (str): Path to the input file.
        output_path (str): Path to save the modified file.
    """
    # Crear un mapeo del 1 al 17 → A-Q
    segment_map = {str(i): string.ascii_uppercase[i - 1] for i in range(1, 18)}

    # Leer el archivo original
    with open(input_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    # Ordenar los números de mayor a menor (para evitar problemas con el 1 y el 10)
    sorted_numbers = sorted(segment_map.keys(), key=int, reverse=True)

    # Reemplazar los segmentos en cada línea
    modified_lines = []
    for line in lines:
        for num in sorted_numbers:  # Asegura que "10" se reemplace antes que "1"
            line = line.replace(f"Segment {num}:", f"Segment {segment_map[num]}:")
            line = line.replace(f"Segment {num}", f"Segment {segment_map[num]}")
            line = line.replace(f"Branch Segment {num}:", f"Branch Segment {segment_map[num]}:")
            line = line.replace(f"Branch Segment {num}", f"Branch Segment {segment_map[num]}")
        modified_lines.append(line)

    # Guardar el archivo corregido
    with open(output_path, "w", encoding="utf-8") as file:
        file.writelines(modified_lines)

    return output_path

##### Read .txt and convert it o a DataFrame

In [ ]:
def parse_and_export_centerlines(file_path):
    branches = defaultdict(list)  # Diccionario donde la clave es el ID de la rama
    current_branch = None  # Guarda el ID de la rama actual
    has_branches = False  # Bandera para detectar si hay múltiples ramas

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            # Detectar una nueva rama
            branch_match = re.match(r"\[New Branch Set\] Branch Segment (\d+):", line)
            if branch_match:
                current_branch = int(branch_match.group(1))
                has_branches = True  # Confirmamos que hay múltiples ramas
                continue

            # Si no hay identificadores de rama, asignamos un ID por defecto
            if current_branch is None:
                current_branch = 0

            # Extraer puntos de la rama actual (ignorar líneas vacías o encabezados)
            if re.match(r"^\s*-?\d+\.\d+", line):
                data = []
                for value in line.split():
                    try:
                        data.append(float(value))  # Convertir a float
                    except ValueError:
                        data.append(None)  # Manejar valores no numéricos como None
                branches[current_branch].append(data)

    # Crear lista de datos para el DataFrame
    all_data = []
    for branch_id, points in branches.items():
        for point in points:
            all_data.append([branch_id] + point)

    # Columnas según el formato esperado
    columns = ["Branch ID", "Px", "Py", "Pz", "Tx", "Ty", "Tz", "Nx", "Ny", "Nz",
               "BNx", "BNy", "BNz", "Dfit", "Dmin", "Dmax", "C", "Dh", "Xh", "Scf", "Area", "E"]

    # Crear DataFrame y retornarlo
    df = pd.DataFrame(all_data, columns=columns)

    return df

In [ ]:
def parse_and_export_centerlines_letters(file_path):
    branches = defaultdict(list)  # Diccionario donde la clave es el ID de la rama
    current_branch = None  # Guarda el ID de la rama actual
    has_branches = False  # Bandera para detectar si hay múltiples ramas

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            # Detectar una nueva rama
            branch_match = re.match(r"\[New Branch Set\] Branch Segment ([A-Z]):", line)
            if branch_match:
                current_branch = branch_match.group(1)
                has_branches = True  # Confirmamos que hay múltiples ramas
                continue

            # Si no hay identificadores de rama, asignamos un ID por defecto
            if current_branch is None:
                current_branch = "A"

            # Extraer puntos de la rama actual (ignorar líneas vacías o encabezados)
            if re.match(r"^\s*-?\d+\.\d+", line):
                data = []
                for value in line.split():
                    try:
                        data.append(float(value))  # Convertir a float
                    except ValueError:
                        data.append(None)  # Manejar valores no numéricos como None
                branches[current_branch].append(data)

    # Crear lista de datos para el DataFrame
    all_data = []
    for branch_id, points in branches.items():
        for point in points:
            all_data.append([branch_id] + point)

    # Columnas según el formato esperado
    columns = ["Branch ID", "Px", "Py", "Pz", "Tx", "Ty", "Tz", "Nx", "Ny", "Nz",
               "BNx", "BNy", "BNz", "Dfit", "Dmin", "Dmax", "C", "Dh", "Xh", "Scf", "Area", "E"]

    # Crear DataFrame y retornarlo
    df = pd.DataFrame(all_data, columns=columns)

    return df

##### Visualize branches

In [ ]:
def visualize_segments(df):
    fig = go.Figure()

    for branch_id in df["Branch ID"].unique():
        branch_points = df[df["Branch ID"] == branch_id]
        fig.add_trace(go.Scatter3d(
            x=branch_points["Px"], y=branch_points["Py"], z=branch_points["Pz"],
            mode='lines',
            name=f"Branch {branch_id}",
            line=dict(width=4)
        ))

    fig.update_layout(
        title="3D Visualization of Centerlines",
        scene=dict(
            xaxis_title="X (Px)",
            yaxis_title="Y (Py)",
            zaxis_title="Z (Pz)"
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        height=700
    )

    fig.show()

In [ ]:
def visualize_segments_letters(df):
    fig = go.Figure()

    for branch_id in df["Branch ID"].unique():
        branch_points = df[df["Branch ID"] == branch_id]
        fig.add_trace(go.Scatter3d(
            x=branch_points["Px"], y=branch_points["Py"], z=branch_points["Pz"],
            mode='lines',
            name=f"Branch {branch_id}",
            line=dict(width=4)
        ))

    fig.update_layout(
        title="3D Visualization of Centerlines",
        scene=dict(
            xaxis_title="X (Px)",
            yaxis_title="Y (Py)",
            zaxis_title="Z (Pz)"
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        height=700
    )

    fig.show()

### **Execution**

##### Parse and export

In [ ]:
### CASE NORMAL 1
rca_path_n1, lca_path_n1 = generate_file_paths('Normal_1')
df_rca_n1 = parse_and_export_centerlines(rca_path_n1)
df_lca_n1 = parse_and_export_centerlines(lca_path_n1)

### CASE NORMAL 7
rca_path_n7, lca_path_n7 = generate_file_paths('Normal_7')
df_rca_n7 = parse_and_export_centerlines(rca_path_n7)
df_lca_n7 = parse_and_export_centerlines(lca_path_n7)

### CASE DISEASED 7
rca_path_d7, lca_path_d7 = generate_file_paths('Diseased_7')
df_rca_d7 = parse_and_export_centerlines(rca_path_d7)
df_lca_d7 = parse_and_export_centerlines(lca_path_d7)

### CASE DISEASED 9
rca_path_d9, lca_path_d9 = generate_file_paths('Diseased_9')
df_rca_d9 = parse_and_export_centerlines(rca_path_d9)
df_lca_d9 = parse_and_export_centerlines(lca_path_d9)

##### Visualize segments with letters

In [ ]:
### LETTERS OUTPUT

# Normal 1
rca_letters_path_n1, lca_letters_path_n1 = generate_file_paths_letters('Normal_1')
rca_letters_n1 = segments_to_letters(rca_path_n1, rca_letters_path_n1)
lca_letters_n1 = segments_to_letters(lca_path_n1, lca_letters_path_n1)
df_letters_rca_n1 = parse_and_export_centerlines_letters(rca_letters_n1)
df_letters_lca_n1 = parse_and_export_centerlines_letters(lca_letters_n1)

# Normal 7
rca_letters_path_n7, lca_letters_path_n7 = generate_file_paths_letters('Normal_7')
rca_letters_n7 = segments_to_letters(rca_path_n7, rca_letters_path_n7)
lca_letters_n7 = segments_to_letters(lca_path_n7, lca_letters_path_n7)
df_letters_rca_n7 = parse_and_export_centerlines_letters(rca_letters_n7)
df_letters_lca_n7 = parse_and_export_centerlines_letters(lca_letters_n7)

# Diseased 7
rca_letters_path_d7, lca_letters_path_d7 = generate_file_paths_letters('Diseased_7')
rca_letters_d7 = segments_to_letters(rca_path_d7, rca_letters_path_d7)
lca_letters_d7 = segments_to_letters(lca_path_d7, lca_letters_path_d7)
df_letters_rca_d7 = parse_and_export_centerlines_letters(rca_letters_d7)
df_letters_lca_d7 = parse_and_export_centerlines_letters(lca_letters_d7)

# Diseased 9
rca_letters_path_d9, lca_letters_path_d9 = generate_file_paths_letters('Diseased_9')
rca_letters_d9 = segments_to_letters(rca_path_d9, rca_letters_path_d9)
lca_letters_d9 = segments_to_letters(lca_path_d9, lca_letters_path_d9)
df_letters_rca_d9 = parse_and_export_centerlines_letters(rca_letters_d9)
df_letters_lca_d9 = parse_and_export_centerlines_letters(lca_letters_d9)

In [ ]:
visualize_segments(df_letters_rca_n1)
visualize_segments(df_letters_lca_n1)

In [ ]:
visualize_segments(df_letters_rca_n7)
visualize_segments(df_letters_lca_n7)

In [ ]:
visualize_segments(df_letters_rca_d7)
visualize_segments(df_letters_lca_d7)

In [ ]:
visualize_segments(df_letters_rca_d9)
visualize_segments(df_letters_lca_d9)

##### Visualize segments with numbers

In [ ]:
### CASE NORMAL 1
rca_path_n1, lca_path_n1 = generate_file_paths('Normal_1')
df_rca_n1 = parse_and_export_centerlines(rca_path_n1)
df_lca_n1 = parse_and_export_centerlines(lca_path_n1)

### CASE NORMAL 7
rca_path_n7, lca_path_n7 = generate_file_paths('Normal_7')
df_rca_n7 = parse_and_export_centerlines(rca_path_n7)
df_lca_n7 = parse_and_export_centerlines(lca_path_n7)

### CASE DISEASED 7
rca_path_d7, lca_path_d7 = generate_file_paths('Diseased_7')
df_rca_d7 = parse_and_export_centerlines(rca_path_d7)
df_lca_d7 = parse_and_export_centerlines(lca_path_d7)

### CASE DISEASED 9
rca_path_d9, lca_path_d9 = generate_file_paths('Diseased_9')
df_rca_d9 = parse_and_export_centerlines(rca_path_d9)
df_lca_d9 = parse_and_export_centerlines(lca_path_d9)

In [ ]:
visualize_segments(df_rca_n1)
visualize_segments(df_lca_n1)

In [ ]:
visualize_segments(df_rca_n7)
visualize_segments(df_lca_n7)

In [ ]:
visualize_segments(df_rca_d7)
visualize_segments(df_lca_d7)

In [ ]:
visualize_segments(df_rca_n1)
visualize_segments(df_lca_n1)